In [ ]:
%matplotlib inline

In [ ]:
import csv
import numpy as np
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from python import IMC, dirtyIMC, Utils
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
data_dir = os.path.abspath(os.path.join('../', 'dataset/'))
ml_dir = os.path.join(data_dir, 'movie_lens/ml-20m/')

In [ ]:
M = 27278
U = 138493
T = 1128
ratio = .1
M = int(M*ratio)
U = int(U*ratio)

Genres = ['Action',
'Adventure',
'Animation',
'Children',
'Comedy',
'Crime',
'Documentary',
'Drama',
'Fantasy',
'Film-Noir',
'Horror',
'Musical',
'Mystery',
'Romance',
'Sci-Fi',
'Thriller',
'War',
'Western',
'IMAX']

In [ ]:
genome_scores_f = open(ml_dir + 'genome-scores.csv', 'rb')
ratings_f = open(ml_dir + 'ratings.csv', 'rb')
movies_f = open(ml_dir + 'movies.csv', 'rb')
tags_f = open(ml_dir + 'tags.csv', 'rb')

genome_scores_r = csv.reader(genome_scores_f, delimiter=',', quotechar='|')
ratings_r = csv.reader(ratings_f, delimiter=',', quotechar='|')
movies_r = csv.reader(movies_f, delimiter=',', quotechar='|')
tags_r = csv.reader(tags_f, delimiter=',', quotechar='|')

In [ ]:
genome_scores_r.next()
GS = np.zeros((M, T))
for row in genome_scores_r:
    if int(row[0]) >= M+1:
        break
    GS[int(row[0])-1, int(row[1])-1] = float(row[2])
    
ratings_r.next()
R = np.zeros((U, M))
Entries = []
for row in ratings_r:
    if int(row[0]) == U+1:
        break
    if int(row[1]) >= M+1:
        continue
    R[int(row[0])-1, int(row[1])-1] = float(row[2])
    Entries.append([int(row[0])-1, int(row[1])-1])

In [ ]:
movies_r.next()
MG = np.zeros((M, len(Genres)))
for row in movies_r:
    if int(row[0]) >= M+1:
        break
    gs = row[-1].split('|')
    for g in gs:
        pos = Genres.index(g)
        MG[int(row[0])-1, pos] = 1.0

In [ ]:
def prepare_train_test(R, Entries, test_ratio=0.2):
    m = int(len(Entries)*test_ratio)
    n1, n2 = R.shape
    Omega = np.random.choice(len(Entries), m, replace=False)
    mask = np.zeros(R.shape)
    for ij in range(m):
        [i, j] = Entries[Omega[ij]]
        mask[i, j] = 1.0

    R_train = R * mask
    return R_train

In [ ]:
"""
Strategy 1: 
X = I
"""
def do_test_imc(k=20, tr=0.2):
    seed = 1
    lamb = 1e-3
    maxiter = 100
    d1 = U
    d2 = len(Genres)
    print 'Generating data...'
    R_train = prepare_train_test(R, Entries, test_ratio=0.2)
    X = np.eye(d1)
    Y = MG
    W0 = np.random.randn(d1, k)
    H0 = np.random.randn(d2, k)

    # Run IMC
    W, H, losses = IMC.IMC(R_train, X, Y, k, lamb, maxiter, W0, H0)

    
#     rms = sqrt(mean_squared_error(y_actual, y_predicted))
    # relerr = norm(W.T.dot(H) - Z, 'fro')**2 / norm(Z, 'fro')**2 * 100
    relerr = norm(X.dot(W.T).dot(H).dot(Y.T) - R, 'fro') ** 2 / norm(A, 'fro') ** 2 * 100
    print 'IMC RelErr = %g' % (relerr)
    # plt.plot(losses)
    # plt.yscale('log')
    # plt.show()

    return relerr

In [ ]:
do_test_imc()